# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import googlemaps
import gmaps
import os
from pprint import pprint
from ipywidgets.embed import embed_minimal_html
from api_keys import g_key

In [3]:
gmaps.configure(api_key=g_key)
maps = googlemaps.Client(key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df = pd.concat((pd.read_csv('output_data/cities_N.csv'), pd.read_csv('output_data/cities_S.csv')), ignore_index=True)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Behshahr,36.69,53.55,82.40,61,75,9.17,IR,1596549401
1,Boende,-0.22,20.87,84.76,60,67,5.17,CD,1596549402
2,Moyamba,8.17,-12.43,81.19,68,92,6.31,SL,1596549402
3,Kahului,20.89,-156.47,75.20,69,90,11.41,US,1596549382
4,Boden,65.83,21.69,64.40,82,40,8.05,SE,1596549403
...,...,...,...,...,...,...,...,...,...
577,Kaputa,-8.47,29.66,82.63,32,0,6.17,ZM,1596549667
578,Ancud,-41.87,-73.82,44.01,100,90,1.12,CL,1596549667
579,Carnarvon,-24.87,113.63,64.40,77,0,14.99,AU,1596549596
580,Olinda,-8.01,-34.86,82.40,54,40,16.11,BR,1596549668


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:

fig = gmaps.figure(layout={'width': '1280px', 'height': '840px'})
heatmap_layer = gmaps.heatmap_layer(df[['Lat', 'Lng']], weights=df['Humidity'])
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='840px', width='1280px'))

In [6]:
# save the output heatmap in html file

embed_minimal_html('output_data/humidity_heatmap.html', views=[fig])

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
df = df.dropna()
df = df[df['Max Temp'] >= 70]
df = df[df['Max Temp'] <= 80]
df = df[df['Wind Speed'] <= 10]
df = df[df['Cloudiness'] == 0]
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
74,Sukhumi,43.00,41.02,79.88,59,0,8.84,GE,1596549439
112,Kamenka,51.32,42.77,78.01,40,0,8.01,RU,1596549457
117,San Vicente,-26.62,-54.13,73.20,48,0,5.73,AR,1596549459
305,Les Herbiers,46.87,-1.02,79.00,33,0,5.82,FR,1596549548
359,San Patricio,19.22,-104.70,79.00,82,0,3.51,MX,1596549354
411,Beloha,-25.17,45.05,77.74,28,0,3.47,MG,1596549595
422,Arrondissement de Saint-Denis,48.92,2.33,78.01,33,0,4.70,FR,1596549600
470,Orán,-23.13,-64.33,72.00,59,0,1.45,AR,1596549620
483,Hoopstad,-27.83,25.91,72.70,17,0,9.46,ZA,1596549626
486,Mapiri,-15.25,-68.17,77.85,42,0,0.25,BO,1596549628


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
def get_hotel_name(row):
    hotels = maps.places(query='hotels nearby', location=(row.Lat, row.Lng), radius=5000)['results']
    return hotels[0]['name']
    # 'Lat' : hotels[0]['geometry']['location']['lat'],
    # 'Lng' : hotels[0]['geometry']['location']['lng'],
    # 'Hotel Name' : hotels[0]['name'],

hotel_df = df
hotel_df['Hotel Name'] = hotel_df.apply(get_hotel_name, axis= 1)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
74,Sukhumi,43.00,41.02,79.88,59,0,8.84,GE,1596549439,"Гостевой дом ""РОЗА"""
112,Kamenka,51.32,42.77,78.01,40,0,8.01,RU,1596549457,Gaba Hotel & Restaurant
117,San Vicente,-26.62,-54.13,73.20,48,0,5.73,AR,1596549459,Gaba Hotel & Restaurant
305,Les Herbiers,46.87,-1.02,79.00,33,0,5.82,FR,1596549548,Château du Boisniard
359,San Patricio,19.22,-104.70,79.00,82,0,3.51,MX,1596549354,Hotel Bahía Melaque
411,Beloha,-25.17,45.05,77.74,28,0,3.47,MG,1596549595,Gaba Hotel & Restaurant
422,Arrondissement de Saint-Denis,48.92,2.33,78.01,33,0,4.70,FR,1596549600,Maison Albar Hotels Le Champs-Elysées
470,Orán,-23.13,-64.33,72.00,59,0,1.45,AR,1596549620,Nuevo Hotel Argentino
483,Hoopstad,-27.83,25.91,72.70,17,0,9.46,ZA,1596549626,Gaba Hotel & Restaurant
486,Mapiri,-15.25,-68.17,77.85,42,0,0.25,BO,1596549628,Gaba Hotel & Restaurant


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = '''
<dl>
    <dt>Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
</dl>
'''
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markerlayer = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markerlayer)

# Display figure
fig

Figure(layout=FigureLayout(height='840px', width='1280px'))

In [11]:
# save the output hotel locations in html file

embed_minimal_html('output_data/hotel_locations.html', views=[fig])